<a href="https://colab.research.google.com/github/rajagopal17/GenAI/blob/main/llama_index_advanced_retrievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load the required libraries

In [ ]:
!pip install llama_index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install --upgrade together
!pip install llama-index-llms-together
!pip install llama-index-embeddings-together

####################################
import textwrap
import llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,SimpleKeywordTableIndex
from llama_index.core import ServiceContext, StorageContext,load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core import download_loader
from llama_index.core.response.notebook_utils import display_response
######################################
#import llama retrievers
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import get_response_synthesizer
######################################
import os
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

######################################
#Provide APIs
import os
from llama_index.core import Settings
from google.colab import userdata
userdata.get('HF_API_TOKEN')
userdata.get('LM_API_KEY')
userdata.get('TG_API_KEY')

HUGGINGFACEHUB_API_TOKEN = "HF_API_TOKEN"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings


Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

import together
together.api_key = userdata.get('TG_API_KEY')


from llama_index.llms.together import TogetherLLM
Settings.llm = TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key)
llm          = Settings.llm





In [2]:
#Load the data and create vector store index and Summary Index

from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex
from llama_index.core.indices import SummaryIndex

#load documents
documents = SimpleDirectoryReader("/content/drive/MyDrive/Assignment9").load_data()


# Create a  vector store index.  VectorStoreIndex only needs an embed model
#index = VectorStoreIndex.from_documents(documents, embed_model=Settings.embed_model)

#Store vectorindex
from llama_index.core.storage.storage_context import StorageContext

#Store VectorStoreIndex to local disk for later use
PERSIST_DIR     = "/content/drive/MyDrive/llama_storage/sapllama2"
#index.storage_context.persist(persist_dir=PERSIST_DIR)
storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)

#reload index from storage
sap_index       = load_index_from_storage(storage_context=storage_context)


#Create a summary index from the same set of documents loaded
summary_index = SummaryIndex.from_documents(documents)


# Summary of llama-index modules
### Vector tool is used in many modules as below:

1. QueryEngineTool - uses (VectorTool and SummaryTool) with single and multiple
2. RouterQuery Engine - uses (VectorTool and SummaryTool)
3. React Agent - uses (vectorTool and SummaryTool)

### Other modules
1. Recursive Retriever with response_mode="tree_summarize"
2. MultistepQuery Engine
3. Hyde Transform Query with prompt template - uses langchain and not discussed here
4. Response Schemas - not discussed here


In the below notebook few of the above modules were discussed with working examples

#Create vector_tool, summary_tool and list_tool. They can be used in different vector based modules

In [3]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


vector_tool = QueryEngineTool(
    sap_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific details and facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing the given content from an entire document."
    )
)


# list_tool is only for retrieving similiar documents from summary index and does not use llm. It is not used for querying with llm.

list_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",use_async=True)

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        '''Useful for questions related to  SAP fixed asset accounting, asset master creation, account determination, field status groups, accounting principles,
           asset capitalization,reporting, depreciation areas,depreciation keys,amortization, year-end processing,balance carryforwards, acquisitions,scrapping, intercompany transfers and retirement '''
    ),
)


In [4]:
#Using vector_tool for querying from the index

vec_query=vector_tool.query_engine.query('As a SAP FICO consultant provide a summary overview for the field depreciation key in asset master in SAP and then give details for each field in depreciation key in bullet points.Please answer based on the provided context only and do not guess').response
wrapped_vec_query = textwrap.fill(vec_query, width=150, break_long_words=False,replace_whitespace=False)
print(wrapped_vec_query)

 The depreciation key in the asset master of SAP FICO module is a crucial field that determines the depreciation run for a specific asset. It is a
customizable feature that can be defined at the chart of depreciation level. The depreciation key consists of several sub-fields that specify the
posting values from the depreciation area to the general ledger and manage certain values.

Here are the details for each field in the depreciation
key:

1. Posting Values from Depreciation Area to General Ledger: This feature allows you to automatically post asset balance sheet values
(APC/proportional value adjustments) and depreciation from each depreciation area to the corresponding general ledger accounts. You can choose whether
values are posted immediately online in Financial Accounting or at periodic intervals. Additionally, you can specify that only depreciation (no APC
values) should be posted automatically.
2. Managing Certain Values: This feature allows you to specify if the depreciatio


#RouterQueryEngine:
## As shown below we can use the vector tools and format the answers in summarized and detailed form:

## We use RouterQueryEngine with the vector tools. RouterQueryEngine wraps both vector & summary tool. And using the bool statement of True or False we instruct whether the query need to be from vector tool or both vector tool and summary tool.

In [5]:
#Single_selector

from llama_index.core.query_engine import RouterQueryEngine

single_query = RouterQueryEngine.from_defaults([vector_tool, summary_tool],select_multi=False)
multi_query  = RouterQueryEngine.from_defaults([vector_tool, summary_tool],select_multi=True)


single_response = single_query.query("what is depreciation key in SAP  from the given context")
multi_response  = multi_query.query('''Can you explain different types of depreciation in SAP and the relevance of
                                       each field in depreciation key.Provide answers from the given context only and do not guess''')

print('Single_response: ')
display_response(single_response)

print('muliple_response')
display_response(multi_response)
#print(f'Summary:\n------------------> \n\n {sigle_response} \n\n Detailed overview:\n----------------->\n\n {multi_response}')

Single_response: 


**`Final Response:`** A depreciation key in SAP is a combination of calculation rules used for determining depreciation amounts. It represents a set of values necessary for determining depreciation, including ordinary depreciation, special depreciation, imputed interest, and scrap value key. These keys are defined at the chart of depreciation level and are available in all company codes. They allow for the uniform setup of the chart of depreciation with special rules for valuation in a specific economic area. Calculation methods are assigned to each depreciation key for the actual determination of depreciation amounts. Control indicators, such as no ordinary depreciation with special depreciation, no interest if no depreciation is planned, period control according to fiscal years, depreciation to the day, no reduction in short year, and assigned calculation methods, are also features of depreciation keys.

muliple_response


**`Final Response:`** In SAP, there are three main types of depreciation: ordinary depreciation, special depreciation, and unplanned depreciation.

Ordinary depreciation is the normal depreciation that occurs in the routine course of business. All depreciation areas are considered as normal depreciation types.

Special depreciation is required for special purposes or parallel valuations, such as tax depreciation or cost depreciation.

Unplanned depreciation is the depreciation provision for unexpected events or losses.

Depreciation areas are the different areas where depreciation can be calculated and applied. Some of the depreciation areas are book depreciation area, special tax depreciation area, net worth tax depreciation, cost depreciation, group depreciation, and investment depreciation.

In SAP, depreciation keys control the automatic calculation of depreciation. These depreciation keys can be modified to define your own methods for calculating asset values. Depreciation keys consist of several fields that are relevant for the calculation of depreciation. However, the context information does not provide specific details about each field in the depreciation key.

Interest calculation for imputed interest in cost accounting is treated as a depreciation type by the system and is controlled by depreciation keys and calculation methods.

The handling of manual depreciation (unplanned depreciation) is explained in detail in Manual Planning of Depreciation.

##Use case for list_tool:  List tool is mainly used to retrieve similiar documents. It uses response_mode='tree_summarize' to collect similiar documents hierarchically. It has no query method. We can use retrieve method to get similiar documents and they can be used in other downstream applications.

In [6]:
list_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",use_async=True)

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        '''Useful for questions related to  SAP fixed asset accounting, asset master creation, account determination, field status groups, accounting principles,
           asset capitalization,reporting, depreciation areas,depreciation keys,amortization, year-end processing,balance carryforwards, acquisitions,scrapping, intercompany transfers and retirement '''
    ),
)
list_query_engine.retrieve("what is account determination")[12].text.replace('\n',"")

'SAP AG  Asset Accou nting (FI- AA)   Asset Accou nting (FI- AA) Asset Accounting Overview  Purpose  The Asset Accou nting (FI-AA) comp onent is use d for managi ng an d sup ervising fixed asset s with the SAP R/3 System. In SAP R/3 Financial Accounting,  it serves as a subsidiary ledger to the FI Gene ral Le dger, providi ng detailed info rmation on tra nsaction s involving fixed assets. Implementation Considerations  The R/3 Asset Accou nting comp onent is intended for i nternatio nal u se in many countrie s, irrespective o f the nature o f the industry.  This  mea ns, for example, that no co untry-spe cific valuation rul es are h ard-co ded in the sy stem. Y ou give this co mponent its co untry-spe cific an d comp any-sp ecific characte r with the settings you ma ke in Cu stomizing. To minim ize the time a nd energy involved in Cu stomi zing, country -specific d efaults are p rovided in the stan dard system where possible. The Impleme ntation Guid e (IMG) provid es the ne cessary fu

In [ ]:
get_list =list_tool.query_engine.retrieve("what is period control ")
get_list=[get_list[i].text for i in range(0,len(get_list))]
#get_list=" ".join(asst_list).replace('\n',"")
get_list

#React Agent: this is another module which uses vector_tool and summary_tool. This module uses chain of thought in response

In [7]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=Settings.llm,
    verbose=True
)

response = agent.chat('''as a SAP consultant can you please explain what are the different types of
                        vendor invoice clearing in SAP from the given context''')

display_response(response)

Thought: The current language of the user is: English. The user is asking about the different types of vendor invoice clearing in SAP. I need to use the vector_search tool to help me answer the question.
Action: vector_search
Action Input: {'input': 'different types of vendor invoice clearing in SAP'}
Observation:  In the provided context, there are a few different types of vendor invoice clearing in SAP that have been mentioned. 

1. Standard Clearing: This function is used when processing open items in accounts payable. It can be used to clear the total down payment to the invoice in F-44 transaction code.

2. Partially Clearing: This function is used when the down payment is more than the invoice amount. The residual amount can be cleared using this function in F-53 transaction code.

3. Residual Clearing: This function is used when the down payment is less than the invoice amount. The residual amount can be cleared using this function in F-53 transaction code.

4. Simulation: This 

**`Final Response:`** The different types of vendor invoice clearing in SAP are Standard Clearing, Partially Clearing, Residual Clearing, Simulation, Alternative Reconciliation Account, Automatic Clearing Account, Special General Ledger Transactions, and Fast Entry Screen (FB10). Each type serves a specific purpose in the clearing process, such as clearing the total down payment to the invoice, checking the clearing of vendor invoices before posting, and using a clearing account for special general ledger transactions.

#Recursive retriever:
###This is interesting module and it uses all the below features for querying

1.   similiarity
2.   llm
3.   Response mode


In [9]:
recur_retriever      = RecursiveRetriever("vector",
                                         retriever_dict={"vector": sap_index.as_retriever(similarity_top_k=5)},
                                         verbose=False)

response_synthesizer = get_response_synthesizer(llm=TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key),
                                             verbose=False,
                                       response_mode="accumulate") # only 'refine',tree_summarize and 'accumulate' are working


# Use above two components to configure query engine
lam_query_engine     = RetrieverQueryEngine.from_args(retriever = recur_retriever,
                                                      llm       = TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key),
                                         response_synthesizer   = response_synthesizer)

print(lam_query_engine.query('''As a SAP FICO consultant please provide a detailed summary of the field depreciation key in asset master in SAP.
                                Please answer based on the provided context only and do not guess''').response)

Response 1:  The depreciation key in the asset master of SAP FICO is a crucial feature that offers additional settings for depreciation calculation. It goes beyond the basic settings already discussed. This key allows for specific configurations that cater to various depreciation rules and regulations of different companies or countries.

One such feature is the option to not calculate ordinary depreciation if special tax depreciation is present. This means that ordinary depreciation is set to zero when special tax depreciation is applicable.

Another feature is the conditional calculation of imputed interest. The system will only calculate imputed interest if depreciation is also being calculated. This is useful for avoiding unnecessary interest calculations.

The depreciation key also allows for period controls to be set according to fiscal years. This means that different period controls can be applied to selected company codes and fiscal years, which can be useful for adhering to s

#Multistep Query Engine

In [11]:
######   Multistep query engine  #######################


from llama_index.core.indices.query.query_transform.base import (StepDecomposeQueryTransform)

# Step decompose transform
step_decompose_transform = StepDecomposeQueryTransform(llm=TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key), verbose=False)


index_summary = "Use to answer questions about gl accounts, customer, vendors, assets, payments, bank accounts,general ledger integration  and invoice clearings and its configurations in SAP"


# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine.multistep_query_engine import (MultiStepQueryEngine)

q_engine        = MultiStepQueryEngine(query_engine    =sap_index.as_query_engine(),
                                       query_transform =step_decompose_transform,
                                       index_summary    =index_summary)


response_llama = q_engine.query("As a SAP FICO consultant provide a summary overview for the field depreciation key in asset master in SAP and then give details for each field in depreciation key in bullet points.Please answer based on the provided context only and do not guess").response
wrapped_response = textwrap.fill(response_llama, width=150, break_long_words=False,replace_whitespace=False)
print(wrapped_response)

 The depreciation key in the asset master of SAP Asset Accounting (FI-AA) is a configuration setting that enables detailed customization of
depreciation calculation. It includes features such as:

- Option to exclude ordinary depreciation if special tax depreciation is present
- Period
controls that can differ from company code and fiscal year specifications
- Depreciation calculation to the day
- Special handling of leap years
-
Depreciation calculation in shortened fiscal years
- Acquisition restrictions to the capitalization year
- Rounding of the percentage rate for
depreciation calculation to a specified number of places

These settings provide adaptability in the calculation of depreciation for various business
needs.

Regarding the fields in the depreciation key:

- Depreciation Key: A unique identifier for the depreciation key
- Useful Life: Specifies the
useful life of the asset in years or months
- Depreciation Start Date: The date from which depreciation is calculated
- Depr

#Combine two different methods


1.   Recursive Retriever with response mode ='accumulate'
2.   Multistep Query Engine



In [14]:
######################################### Recursive retriever ############################

recur_retriever      = RecursiveRetriever("vector",
                                         retriever_dict={"vector": sap_index.as_retriever(similarity_top_k=5)},
                                         verbose=False)

response_synthesizer = get_response_synthesizer(llm=TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key),
                                             verbose=False,
                                       response_mode="tree_summarize") # only 'refine' and 'accumulate' are working


# Use above two components to configure query engine
lam_query_engine     = RetrieverQueryEngine.from_args(retriever = recur_retriever,
                                                      llm       = TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key),
                                         response_synthesizer   = response_synthesizer)


acc_str = lam_query_engine.query('as a SAP consultant can you please provide a high level summary of integration of material management with financial accounting. Provide answers only from the given context and do not guess')

######################################## Multistep Query Engine ############################

from llama_index.core.indices.query.query_transform.base import (StepDecomposeQueryTransform)

# Step decompose transform
step_decompose_transform = StepDecomposeQueryTransform(llm=TogetherLLM(model="mistralai/Mixtral-8x7B-Instruct-v0.1", api_key=together.api_key), verbose=True)


index_summary = acc_str.response #   Index_summary is derived from recursive retriever method above

# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine.multistep_query_engine import (MultiStepQueryEngine)

q_engine        = MultiStepQueryEngine(query_engine     = sap_index.as_query_engine(),
                                       query_transform  = step_decompose_transform,
                                       index_summary    = index_summary)


In [ ]:
response_llama   = q_engine.query("Can you provide step by step details how goods movement in material management are integrated with financial accounting using transaction keys, movement types etc. Provide answer only if it is in the given context").response  #Provide answer based on given context only and do not answer if not in the given context. Do not guess")
wrapped_response = textwrap.fill(response_llama, width=150, break_long_words=False,replace_whitespace=False)
wrapped_str = textwrap.fill(acc_str.response, width=150, break_long_words=False,replace_whitespace=False)

> Current query: Can you provide step by step details how goods movement in material management are integrated with financial accounting using transaction keys, movement types etc. Provide answer only if it is in the given context
> New query:  How is the integration of general ledger accounts achieved during the creation of a material master record for the integration of financial accounting and material management in SAP?
> Current query: Can you provide step by step details how goods movement in material management are integrated with financial accounting using transaction keys, movement types etc. Provide answer only if it is in the given context
> New query:  How are general ledger accounts integrated during the creation of a material master record for the integration of financial accounting and material management in SAP using transaction keys and movement types?
> Current query: Can you provide step by step details how goods movement in material management are integrated with fi

In [ ]:
print(f"<----------->\nSummary: \n\n{wrapped_str}\n\n <---------->  \n Detailed analysis:\n\n {wrapped_response}")

<----------->
Summary: 

 Integration of material management with financial accounting in SAP is a complex activity during the implementation project. The finance module is
linked up with most of the other modules, including material management. This integration is done automatically for intra-module activities, but
inter-module integration is a challenging task for the implementation team. The FICO module consultant plays a significant role in this process.
Specifically, for the integration of the FICO module with material management, general ledger accounts are required to integrate FI and MM
transactions with the necessary parameters. Changes to the automatic determination of accounts can be made in OBYC. The creation of a material master
record involves several steps, including giving industry sector, material type, plant, and storage location, and selecting views such as basic data 1,
purchasing, and accounting 1. Valuation class and price control are important aspects of accounti